# Create Directories For Cellranger Outputs

In [4]:
import pandas as pd
import os

In [5]:
root = '/om/scratch/Mon/mabdel03/Tsai/Counts'
ACE_df = pd.read_csv('/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/FASTQ_Transfer/CSVs/Corrected_TsaiACE_ToOpenmind.csv')

projids = set(ACE_df['projid'])

for num in projids:
    os.mkdir(os.path.join(root, str(num)))

FileExistsError: [Errno 17] File exists: '/om/scratch/Mon/mabdel03/Tsai/Counts/26569730'

In [6]:
ACE_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Batch,projid,Library_ID,Tsai_path,Openmind_Destination
0,0,0,9,last_16,24680888,D19-10914_10,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/Tsai/FASTQs/24680888/...
1,1,1,10,last_16,22789958,D19-10914_11,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/Tsai/FASTQs/22789958/...
2,2,2,11,last_16,7265221,D19-10914_12,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/Tsai/FASTQs/7265221/D...
3,3,3,12,last_16,51520126,D19-10914_13,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/Tsai/FASTQs/51520126/...
4,4,4,13,last_16,94144536,D19-10914_14,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/Tsai/FASTQs/94144536/...


# Generate batch scripts for Cellranger count

In [7]:
import subprocess

In [9]:
parent_scripts = '/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/Counts/Batch_Scripts'
for num in projids:
    ACE_sub = ACE_df[ACE_df['projid'] == num]
    ACE_sub = ACE_sub.reset_index(drop=True)

    libID = ACE_sub['Library_ID'][0]
    fastqs = ''

    if len(ACE_sub['projid']) == 2:
        fastqs = f'{ACE_sub['Openmind_Destination'][0]},{ACE_sub['Openmind_Destination'][1]}'
    else:
        fastqs = ACE_sub['Openmind_Destination'][0]

    out_dir = os.path.join(root, str(num))

    output = f"""#!/bin/bash
#SBATCH -t 47:00:00
#SBATCH -n 32
#SBATCH --mem=128G
#SBATCH --output=/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/Counts/Batch_Scripts_Outs/slurm-%j.out
#SBATCH --error=/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/Counts/Batch_Scripts_Err/slurm-%j.err
#SBATCH --mail-user=mabdel03@mit.edu
#SBATCH --mail-type=FAIL
export PATH=/om2/user/$USER/apps/yard/cellranger-8.0.0:$PATH
cellranger count --create-bam false --include-introns true --nosecondary --r1-length 26 --id {libID} --transcriptome=/orcd/data/lhtsai/001/om2/mabdel03/yard/references/human/refdata-gex-GRCh38-2020-A --sample {libID} --fastqs {fastqs} --output-dir={out_dir}
"""
    filename = str(num)+'_count.sh' #name the file for the count batch script --> libID_count.sh
    f = open(os.path.join(parent_scripts, filename), 'x') #make the file
    f.write(output) #write to the file
    f.close()
    sbatch_command = f'sbatch {os.path.join(parent_scripts, filename)}' #submit sbatch command
    process = subprocess.Popen(sbatch_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    
    